In [56]:
# Imports
import random
import math
import matplotlib.pyplot as plt

## Helper functions

In [57]:
def ReLU(x):
    if x < 0:
        return 0
    return x

def sigmoid(x):
    return 1/(1 + math.exp(-x))

def topological_sort(edges):
        visited = set()
        order = []

        def visit(n):
            if n in visited:
                return
            visited.add(n)
            for m in edges[n]:
                visit(m)
            order.append(n)

        for node in edges:
            visit(node)

        return order[::-1]

## NEAT Algorithm

Original paper: https://nn.cs.utexas.edu/downloads/papers/stanley.ec02.pdf


#### Network Encoding

In the NEAT algorithm each neuron in the neural network is represented as:

![image](assets/genotype.png)

*Source: [Evolving Neural Networks through Augmenting Topologies](https://nn.cs.utexas.edu/downloads/papers/stanley.ec02.pdf)*


#### Mutation

There are 2 types of mutation:
- Add connection
- Add node

![image](assets/mutation.png)

*Source: [Evolving Neural Networks through Augmenting Topologies](https://nn.cs.utexas.edu/downloads/papers/stanley.ec02.pdf)*


#### Crossover

![image](assets/mutation.png)

*Source: [Evolving Neural Networks through Augmenting Topologies](https://nn.cs.utexas.edu/downloads/papers/stanley.ec02.pdf)*


In [58]:
class InnovationTracker:
    def __init__(self):
        self.current_innovation = 0
        self.connection_innovations = {}  # (in_node_id, out_node_id) -> innovation_number
        self.node_innovations = {}        # connection_innovation -> (node_innovation, conn1_innovation, conn2_innovation)
        self.node_id_counter = 0
    
    def get_connection_innovation(self, in_node_id, out_node_id):
        """Get innovation number for a connection, creating new if needed"""
        key = (in_node_id, out_node_id)
        if key not in self.connection_innovations:
            self.connection_innovations[key] = self.current_innovation
            self.current_innovation += 1
        return self.connection_innovations[key]
    
    def get_node_innovation(self, connection_innovation):
        """Get innovation numbers for node insertion, creating new if needed"""
        if connection_innovation not in self.node_innovations:
            # Create new node and two connections
            node_id = self.node_id_counter
            self.node_id_counter += 1
            
            conn1_innovation = self.current_innovation
            self.current_innovation += 1
            
            conn2_innovation = self.current_innovation
            self.current_innovation += 1
            
            self.node_innovations[connection_innovation] = (node_id, conn1_innovation, conn2_innovation)
        
        return self.node_innovations[connection_innovation]


class NodeGene:
    def __init__(self, id, layer, activation, bias):
        self.id = id
        self.layer = layer  # The layer to which the node belongs
        self.activation = activation    # Activation function
        self.bias = bias
    
    def __eq__(self, other):
        return isinstance(other, NodeGene) and self.id == other.id
    
    def __hash__(self):
        return hash(self.id)
    
    def copy(self):
        return NodeGene(self.id, self.layer, self.activation, self.bias)


class ConnectionGene:
    def __init__(self, in_node_id: int, out_node_id: int, weight: float,  innov: int, enabled: bool = True):
        self.in_node_id = in_node_id
        self.out_node_id = out_node_id
        self.weight = weight
        self.enabled = enabled  # Whether the connection is enabled or not
        self.innov = innov  # Innovation number described in the paper
    
    def copy(self):
        return ConnectionGene(self.in_node_id, self.out_node_id, self.weight, self.enabled, self.innov)


class Genome:
    def __init__(self, nodes, connections):
        self.nodes = {node.id: node for node in nodes}
        self.connections = connections
        self.fitness = 0
    
    def get_node(self, node_id):
        return self.nodes.get(node_id, None)
    
    def mutate_add_connection(self, innov: InnovationTracker):
        node_list = list(self.nodes.values())
        node1, node2 = random.sample(node_list, 2)

        # Check if it's creating a loop to the same node
        if node1 == node2:
            return
        # Check if it's creating a connection between two nodes on the same layer
        if node1.layer == node2.layer:
            return

        # Check whether the connection already exists
        for c in self.connections:
            if c.in_node_id == node1.id and c.out_node_id == node2.id:
                return
        
        innov_num = innov.get_connection_innovation(node1.id, node2.id)
        new_conn = ConnectionGene(node1.id, node2.id, random.uniform(-1, 1), innov_num, True)
        self.connections.append(new_conn)
    
    def mutate_add_node(self, innov: InnovationTracker):
        connection = random.choice(self.connections)
        connection.enabled = False  # Disable the connection

        node_id, conn1_innov, conn2_innov = innov.get_node_innovation(connection.innov) 

        # Create node and connections
        new_node = NodeGene(node_id, "hid", ReLU, random.uniform(-1,1))
        conn1 = ConnectionGene(connection.in_node_id, node_id, 1, conn1_innov, True)
        conn2 = ConnectionGene(node_id, connection.out_node_id, connection.weight, conn2_innov, True)

        self.nodes[node_id] = new_node
        self.connections.extend([conn1, conn2])
    
    def mutate_weights(self, rate=0.8, power=0.5):
        for conn in self.connections:
            if random.random() < rate:
                if random.random() < 0.1:
                    conn.weight = random.uniform(-1, 1)
                else:
                    conn.weight += random.gauss(0, power)
                    conn.weight = max(-5, min(5, conn.weight))  # Clamp weights
    
    def mutate_bias(self, rate=0.7, power=0.5):
        for node in self.nodes.values():
            if node.layer != "input" and random.random() < rate:
                if random.random() < 0.1:
                    node.bias = random.uniform(-1, 1)
                else:
                    node.bias += random.gauss(0, power)
                    node.bias = max(-5, min(5, node.bias))

    def mutate(self, innov, conn_mutation_rate=0.05, node_mutation_rate=0.03, weight_mutation_rate=0.8, bias_mutation_rate=0.7):
        self.mutate_weights(weight_mutation_rate)
        self.mutate_bias(bias_mutation_rate)

        if random.random() < conn_mutation_rate:
            self.mutate_add_connection(innov)
        
        if random.random() < node_mutation_rate:
            self.mutate_add_node(innov)
    
    def evaluate(self, input_values: list[float]):
        node_values = {}
        node_inputs = {n: [] for n in self.nodes}

        input_nodes = [n for n in self.nodes.values() if n.layer == "input"]
        output_nodes = [n for n in self.nodes.values() if n.layer == "output"]

        # Verify that the number of input values matches the number of input nodes
        if len(input_nodes) != len(input_values):
            raise ValueError(f"Number of inputs doesn't match number of input nodes.")
        
        # Assign input values
        for node, val in zip(input_nodes, input_values):
            node_values[node.id] = val

        edges = {}
        for n in self.nodes.values():
            edges[n] = []
        
        for conn in self.connections:
            if conn.enabled:
                in_node = self.get_node(conn.in_node_id)
                out_node = self.get_node(conn.out_node_id)
                edges[in_node].append(out_node)
                node_inputs[conn.out_node_id].append(conn)
        
        sorted_nodes = topological_sort(edges)

        for node in sorted_nodes:
            if node.id in node_values:
                continue
            
            incoming = node_inputs[node.id]
            total_input = sum(
                node_values[c.in_node_id] * c.weight for c in incoming
            ) + node.bias

            node_values[node.id] = node.activation(total_input)

        return [node_values.get(n.id, 0) for n in output_nodes]


In [59]:
def crossover(parent1: Genome, parent2: Genome) -> Genome:
    """ Crossover assuming parent1 is the fittest parent """
    # Collect all nodes from both parents
    all_nodes = {}
    for node in parent1.nodes.values():
        all_nodes[node.id] = node.copy()
    for node in parent2.nodes.values():
        all_nodes[node.id] = node.copy()        

    # Build maps of genes keyed by innovation number
    genes1 = {g.innov: g for g in parent1.connections}
    genes2 = {g.innov: g for g in parent2.connections}

    offspring_connections = []
    offspring_nodes = set()

    # Combine all innovation numbers
    all_innovs = set(genes1.keys()) | set(genes2.keys())

    for innov in sorted(all_innovs):
        gene1 = genes1.get(innov)
        gene2 = genes2.get(innov)
        
        if gene1 and gene2:  # Matching genes
            selected = random.choice([gene1, gene2])
            gene_copy = selected.copy()

            if not gene1.enabled or not gene2.enabled:  # 75% chance of the offsprign gene being disabled
                if random.random() < 0.75:
                    selected.enabled = False

        elif gene1 and not gene2:   # Disjoint gene (from the fittest parent)
            gene_copy = gene1.copy()
        
        else:   # Not taking disjoint genes from less fit parent
            continue
        
        # get nodes
        in_node = all_nodes.get(gene_copy.in_node_id)
        out_node = all_nodes.get(gene_copy.out_node_id)
        
        offspring_connections.append(gene_copy)
        offspring_nodes.add(in_node)
        offspring_nodes.add(out_node)

    offspring_nodes = list(offspring_nodes) # Remove the duplicates
    return Genome(offspring_nodes, offspring_connections)


def distance(genome1: Genome, genome2: Genome, c1=1.0, c2=1.0, c3=0.4):
    genes1 = {g.innov: g for g in genome1.connections}
    genes2 = {g.innov: g for g in genome2.connections}

    innovations1 = set(genes1.keys())
    innovations2 = set(genes2.keys())

    matching = innovations1 & innovations2
    disjoint = (innovations1 ^ innovations2)
    excess = set()

    max_innov1 = max(innovations1) if innovations1 else 0
    max_innov2 = max(innovations2) if innovations2 else 0
    max_innov = min(max_innov1, max_innov2)

    # Separate excess from disjoint
    for innov in disjoint.copy():
        if innov > max_innov:
            excess.add(innov)
            disjoint.remove(innov)

    # Weight difference of matching genes
    if matching:
        weight_diff = sum(
            abs(genes1[i].weight - genes2[i].weight) for i in matching
        )
        avg_weight_diff = weight_diff / len(matching)
    else:
        avg_weight_diff = 0

    # Normalize by N
    N = max(len(genome1.connections), len(genome2.connections))
    if N < 20:  # NEAT typically uses 1 if N < 20
        N = 1

    delta = (c1 * len(excess)) / N + (c2 * len(disjoint)) / N + c3 * avg_weight_diff
    return delta


In [60]:
class Species:
    def __init__(self, representative: Genome):
        self.representative = representative
        self.members = [representative]
        self.adjusted_fitness = 0
        self.best_fitness = -float('inf')
        self.stagnant_generations = 0
    
    def add_member(self, genome: Genome):
        self.members.append(genome)
    
    def clear_members(self):
        self.members = []
    
    def update_fitness_stats(self):
        if self.members:
            self.max_fitness = max(member.fitness for member in self.members)
            self.adjusted_fitness = sum(member.fitness for member in self.members) / len(self.members)


class Speciator:
    def __init__(self, compatibility_threshold=3.0):
        self.species = []
        self.compatibility_threshold = compatibility_threshold
    
    def speciate(self, population: list[Genome]):
        """ Group genomes into species based on distance """
        # Clear all species for the new generation
        for s in self.species:
            s.clear_members()
        
        for genome in population:
            found_species = False
            for species in self.species:
                if distance(genome, species.representative) < self.compatibility_threshold:
                    species.add_member(genome)
                    found_species = True
                    break
            
            if not found_species:
                new_species = Species(representative=genome)
                self.species.append(new_species)

        # Remove empty species
        self.species = [s for s in self.species if s.members]

        # Recompute adjusted fitness
        for species in self.species:
            species.update_fitness_stats()
            # Update representative to be the best member
            species.representative = max(species.members, key=lambda g: g.fitness)

    def get_species(self):
        return self.species


In [61]:
def create_initial_genome(num_inputs, num_outputs, innov):
    input_nodes = []
    output_nodes = []
    connections = []
    
    # Create input nodes
    for i in range(num_inputs):
        node = NodeGene(i, "input", lambda x: x, 0)
        input_nodes.append(node)
    
    # Create output nodes
    for i in range(num_outputs):
        node_id = num_inputs + i    # We start the ids where we left in the previous loop
        node = NodeGene(node_id, "output", sigmoid, random.uniform(-1, 1))
        output_nodes.append(node)
    
    # Create connections
    for i in range(num_inputs):
        for j in range(num_outputs):
            in_node_id = i
            out_node_id = j
            innov_num = innov.get_connection_innovation(in_node_id, out_node_id)
            weight = random.uniform(-1, 1)
            conn = ConnectionGene(in_node_id, out_node_id, weight, innov_num, True)
            connections.append(conn)
    
    
    all_nodes = input_nodes + output_nodes
    return Genome(all_nodes, connections)


def create_initial_population(size, num_inputs, num_outputs, innov):
    population = []
    for _ in range(size):
        genome = create_initial_genome(num_inputs, num_outputs, innov)
        population.append(genome)
    return population


def tournament_selection(population, fitness_scores, tournament_size=3):
    """Tournament selection"""
    def tournament():
        contestants = random.sample(list(zip(population, fitness_scores)), tournament_size)
        winner = max(contestants, key=lambda x: x[1])
        return winner[0]
    
    return tournament(), tournament()


def reproduce_species(species, offspring_count, innov):
    """Reproduce within a species"""
    offspring = []
    
    if len(species.members) == 1:
        # Only one member
        for _ in range(offspring_count):
            child = Genome(species.members[0].nodes[:], 
                         [c.copy() for c in species.members[0].connections])
            child.mutate(innov)
            offspring.append(child)
    else:
        # Multiple members, use crossover
        for _ in range(offspring_count):
            parent1, parent2 = random.sample(species.members, 2)
            
            # Ensure parent1 is fitter
            if parent1.fitness < parent2.fitness:
                parent1, parent2 = parent2, parent1
            
            child = crossover(parent1, parent2)
            child.mutate(innov)
            offspring.append(child)
    
    return offspring

### XOR Problem

In [62]:
# XOR Problem data
X = [[0, 0], [0, 1], [1, 0], [1, 1]]
y = [0, 1, 1, 0]

def fitness_xor(genome):
    """Calculate fitness for XOR problem"""
    total_error = 0
    for i in range(len(X)):
        try:
            output = genome.evaluate(X[i])
            print(f"Output: {output}")
            if output:
                error = abs(output[0] - y[i])
                total_error += error
            else:
                error = y[i]
                total_error += error
        except Exception as e:
            print(f"Error: {e}")
            return 0  # Return 0 fitness if evaluation fails
    
    fitness = 4 - total_error
    return max(0, fitness)


In [63]:
def evolution(population, fitness_scores, speciator, innov):
    new_population = []
    
    # Assign fitness to genomes
    for genome, fitness in zip(population, fitness_scores):
        genome.fitness = fitness
    
    # Speciate population
    speciator.speciate(population)
    species_list = speciator.get_species()
    print(f"Species created: {len(species_list)}")

    total_adjusted_fitness = sum(s.adjusted_fitness for s in species_list)
    print(f"Total adjusted fitness: {total_adjusted_fitness}")

    # elitism
    for species in species_list:
        if species.members:
            best_genome = max(species.members, key=lambda g: g.fitness)
            new_population.append(best_genome)

    remaining_offspring = len(population) - len(new_population)

    # Allocate the remaining offspring
    for species in species_list:
        if total_adjusted_fitness > 0:
            offspring_count = int((species.adjusted_fitness / total_adjusted_fitness) * remaining_offspring)    # The more fit species will have more offspring
        else:
            offspring_count = remaining_offspring // len(species_list)  # If all the species performed poorly, assign offspring evenly between them
        
        if offspring_count > 0:
            offspring = reproduce_species(species, offspring_count, innov)
            new_population.extend(offspring)
    
    # Ensure there are enough individuals (we could have less because of the rounding error)
    while len(new_population) < len(population):
        best_species = max(species_list, key=lambda s: s.adjusted_fitness)
        offspring = reproduce_species(best_species, 1, innov)
        new_population.extend(offspring)

    return new_population



In [64]:
def run_neat_xor(generations=50, pop_size=50, target_fitness=3.9, speciator_threshold=3.0):
    NUM_INPUTS = 2
    NUM_OUTPUTS = 1
    
    # Initialize Innovation Number and Speciator
    innov = InnovationTracker()
    speciator = Speciator(speciator_threshold)

    # Create initial population
    population = create_initial_population(pop_size, NUM_INPUTS, NUM_OUTPUTS, innov)
    
    # Stats
    best_fitness_history = []
    avg_fitness_history = []
    species_count_history = []
    
    # main loop
    for gen in range(generations):
        fitness_scores = [fitness_xor(g) for g in population]

        print(f"fitness: {fitness_scores}")
        
        # get the stats
        best_fitness = max(fitness_scores)
        avg_fitness = sum(fitness_scores) / len(fitness_scores)
        # fitness_dict = {genome: fitness for genome, fitness in zip(population, fitness_scores)}
        # speciator.speciate(population, fitness_dict)
        # species_count = len(speciator.get_species())

        best_fitness_history.append(best_fitness)
        avg_fitness_history.append(avg_fitness)
        # species_count_history.append(species_count)

        # print(f"Generation {gen}: Best={best_fitness}, Avg={avg_fitness}, Species={species_count}")
        print(f"Generation {gen}: Best={best_fitness}, Avg={avg_fitness}")
        
        # Check if we achieved the target fitness
        if best_fitness >= target_fitness:
            print(f"Problem was solved in {gen} generations")
            print(f"Best fitness achieved: {max(best_fitness_history)}")
            
            best_genome = population[fitness_scores.index(best_fitness)]
            return best_genome, best_fitness_history, avg_fitness_history, species_count_history
        
        population = evolution(population, fitness_scores, speciator, innov)

    
    print(f"Couldn't solve the XOR problem in {generations} generations")
    print(f"Best fitness achieved: {max(best_fitness_history)}")

    return None, best_fitness_history, avg_fitness_history, species_count_history




In [65]:
best_genome, best_fitness_history, avg_fitness_history, species_count_history = run_neat_xor(2, 5)


Output: [0.6566837856777289]
Output: [0.6566837856777289]
Output: [0.6566837856777289]
Output: [0.6566837856777289]
Output: [0.37263266032020703]
Output: [0.37263266032020703]
Output: [0.37263266032020703]
Output: [0.37263266032020703]
Output: [0.2893389035144679]
Output: [0.2893389035144679]
Output: [0.2893389035144679]
Output: [0.2893389035144679]
Output: [0.5178639420849918]
Output: [0.5178639420849918]
Output: [0.5178639420849918]
Output: [0.5178639420849918]
Output: [0.7284985833753003]
Output: [0.7284985833753003]
Output: [0.7284985833753003]
Output: [0.7284985833753003]
fitness: [2.0, 2.0, 2.0, 2.0, 2.0]
Generation 0: Best=2.0, Avg=2.0
Species created: 1
Total adjusted fitness: 2.0
Output: [0.6566837856777289]
Output: [0.6566837856777289]
Output: [0.6566837856777289]
Output: [0.6566837856777289]
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
Output: []
fitness: [